In [55]:
# Database stuff
from google.cloud.sql.connector import Connector
import sqlalchemy
from getpass import getpass
import pg8000
import google.auth as auth


import pandas as pd


In [56]:
# Load our credentials from the supplied service account key file
creds = auth.load_credentials_from_file('useful-music-251918-b8ecc9855a02.json')
# Credentials are retrieved as a tuple object with credential object and project name
creds[0]

In [81]:
# Use getpass to get the database password
password = getpass('Enter Password')

# build connection
# Connector is a Google object that manages the creation of our connector string
def get_conn() -> pg8000.native.Connection:
    with Connector(credentials=creds[0]) as connector:
        conn = connector.connect(
            # Project name:region:database name
            "useful-music-251918:us-central1:project-db",
            # Driver
            "pg8000",
            user="postgres",
            password=password,
            db="postgres"
        )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=get_conn,
    echo_pool='debug',
    echo=False,
)

Enter Password ········


In [67]:
pool

Engine(postgresql+pg8000://)

In [71]:
# Create an iterator to break up the flows data into chunks
pseof_df = pd.read_csv('https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseof_all.csv', chunksize=10000, low_memory=False)
pseof_df

In [82]:
# Create an iterator to break up the earnings data into chunks
pseoe_df = pd.read_csv('https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseoe_all.csv', chunksize=10000, low_memory=False)
pseoe_df

In [78]:
# Take our iterator, table name, and sqlalchemy connection to upload our data
def push_to_db(df_iter, table, conn):
    rows_imported = 0
    for data in df_iter:

        print(f'importing rows {rows_imported} to {rows_imported + len(data)}...\n', end='')
        data.to_sql(name=table, con=conn, if_exists='append', method='multi')
        rows_imported += len(data)

        print(f'Done.')

In [83]:
push_to_db(df_iter=pseoe_df, table='pseoe', conn=pool)

importing rows 0 to 10000...
2022-05-20 10:14:46,722 DEBUG sqlalchemy.pool.impl.QueuePool Created new connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00>
2022-05-20 10:14:47,086 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> checked out from pool
2022-05-20 10:14:47,088 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> being returned to pool
2022-05-20 10:14:47,090 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> rollback-on-return
2022-05-20 10:14:47,092 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> checked out from pool
2022-05-20 10:14:47,266 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> being returned to pool
2022-05-20 10:14:47,268 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Co

error: 'h' format requires -32768 <= number <= 32767

In [61]:
push_to_db(df_iter=pseof_df, table='pseof', conn=pool)

C:\Users\zach1\AppData\Local\Temp\ipykernel_21436\1629995334.py:4: DtypeWarning: Columns (2,4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  for data in df_iter:


importing rows 0 to 100000...

Exception closing connection <pg8000.dbapi.Connection object at 0x0000023E2B7019C0>
Traceback (most recent call last):
  File "C:\Users\zach1\Coding\env\mlenv\lib\site-packages\pg8000\core.py", line 771, in handle_messages
    code, data_len = ci_unpack(self._read(5))
struct.error: unpack_from requires a buffer of at least 5 bytes for unpacking 5 bytes at offset 0 (actual buffer size is 0)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\zach1\Coding\env\mlenv\lib\site-packages\sqlalchemy\engine\base.py", line 1799, in _execute_context
    self.dialect.do_executemany(
  File "C:\Users\zach1\Coding\env\mlenv\lib\site-packages\sqlalchemy\engine\default.py", line 729, in do_executemany
    cursor.executemany(statement, parameters)
  File "C:\Users\zach1\Coding\env\mlenv\lib\site-packages\pg8000\dbapi.py", line 492, in executemany
    self.execute(operation, parameters)
  File "C:\Users\zach1\Coding\env\mlenv\lib\sit

InterfaceError: (pg8000.exceptions.InterfaceError) network error
[SQL: INSERT INTO pseof (index, agg_level_pseo, inst_level, institution, degree_level, cip_level, cipcode, grad_cohort, grad_cohort_years, geo_level, geography, ind_level, industry, y1_grads_emp, y1_grads_emp_instate, y5_grads_emp, y5_grads_emp_instate, y10_grads_emp, y10_grads_emp_instate, y1_grads_nme, y5_grads_nme, y10_grads_nme, status_y1_grads_emp, status_y1_grads_emp_instate, status_y5_grads_emp, status_y5_grads_emp_instate, status_y10_grads_emp, status_y10_grads_emp_instate, status_y1_grads_nme, status_y5_grads_nme, status_y10_grads_nme) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)]
[parameters: ((0, 38, 'I', '00105100', 5, 'A', 0, 0, 3, 'N', 0, 'A', 0, 49897.0, 27741.0, 32189.0, 18740.0, 20282.0, 12062.0, 30983.0, 10235.0, 7281.0, 1, 1, 1, 1, 1, 1, 1, 1, 1), (1, 38, 'I', '00105100', 7, 'A', 0, 0, 5, 'N', 0, 'A', 0, 14182.0, 8311

In [84]:
pseoe_df = pd.read_csv('https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseoe_all.csv', low_memory=False)

In [85]:
pseoe_df.to_sql(name='pseoe', con=pool, if_exists='append', method='multi')

2022-05-20 10:17:43,350 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> checked out from pool
2022-05-20 10:17:43,351 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> being returned to pool
2022-05-20 10:17:43,352 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> rollback-on-return
2022-05-20 10:17:43,352 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> checked out from pool
2022-05-20 10:17:43,526 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> being returned to pool
2022-05-20 10:17:43,529 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0000023E5D684D00> rollback-on-return
2022-05-20 10:17:44,325 DEBUG sqlalchemy.pool.impl.QueuePool Connection <pg8000.dbapi.Connection object at 0x0

error: 'h' format requires -32768 <= number <= 32767

In [ ]:
pseoe_df = pd.read_csv('https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseoe_all.csv', low_memory=False)

In [ ]:
pseof_df = pd.read_csv('https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseof_all.csv', low_memory=False)

In [ ]:
pseof_df.to_sql(name='pseof', con=pool, if_exists='append', method='multi')